**Ejercicio 1:** Utilizando la descomposición LU de esta práctica y las sustituciones progresiva y regresiva de la práctica anterior, resuelve el sistema $A x=b$ donde
$$A=\left(\begin{array}{rrr}
1 & 4 & 1 \\
1 & 3 & 5 \\
-2 & 0 & 1 \end{array} \right) \qquad b=\left(2,-5,1 \right) $$ 

In [5]:
import math
import numpy as np
# Primero copiaré la función de sustitución progresiva y regresiva de la práctica anterior
def sust_progresiva(A,b):
    """
    Esta función resuelve el sistema Ax=b mediante sustitución progresiva. 
    A y b deben ser arrays de numpy. 
    A debe ser una matriz triangular inferior.
    """
    A=A.astype(float)     # cambiamos los elementos a tipo flotante
    b=b.astype(float) 
    n=np.size(b)
    x=np.zeros(n)         # almacenaremos aquí la solución del sistema                  
    # ahora se resuelve el sistema por sustitución regresiva
    x[0]=b[0]/A[0,0]
    for i in range(1,n):
        restar = 0
        for j in range(0,i):
            restar += (x[j]*A[i,j])
        x[i]=(b[i]- restar)/A[i,i]
                
    return x

def sust_regresiva(A,b):
    """
    Esta función resuelve el sistema Ax=b mediante sustitución regresiva. 
    A y b deben ser arrays de numpy. 
    A debe ser una matriz triangular superior.
    """
    A=A.astype(float)     # cambiamos los elementos a tipo flotante
    b=b.astype(float) 
    n=np.size(b)
    x=np.zeros(n)         # almacenaremos aquí la solución del sistema
                       
    # ahora se resuelve el sistema por sustitución regresiva
    x[n-1]=b[n-1]/A[n-1,n-1]
    for i in range(n-2,-1,-1):     
        x[i]=(b[i]-np.dot(A[i][i+1:],x[i+1:]))/A[i,i]
                
    return x

In [6]:
def LU_desc(A):
    """
    Esta función calcula la descomposición LU de Doolittle de una matriz A sin intercambio de filas.
    A debe ser un array de Numpy.
    """
    n=np.shape(A)[1]
    U=np.zeros([n,n])
    L=np.eye(n)
    
    for k in range(n-1):
        U[k,k]=A[k,k]-np.dot(L[k,:k],U[:k,k])
        for j in range(k+1,n):
            U[k,j]=A[k,j]-np.dot(L[k,:k],U[:k,j])
        for j in range(k+1):
            L[k+1,j]=(A[k+1,j]-np.dot(L[k+1,:j],U[:j,j]))/U[j,j]
    U[n-1,n-1]=A[n-1,n-1]-np.dot(L[n-1,:n-1],U[:n-1,n-1])
    return L, U

In [8]:
A = np.array([[1,4,1],[1,3,5],[-2,0,1]])
b = np.array([2,-5,1])
L,U = LU_desc(A)
y = sust_progresiva(L,b)
x = sust_regresiva(U,y)

print("Solución =", x)
print("Comprobación =", np.linalg.solve(A,b))

Solución = [-1.22857143  1.17142857 -1.45714286]
Comprobación = [-1.22857143  1.17142857 -1.45714286]


**Ejercicio 2:** Define una función que resuelva un sistema de ecuaciones lineales mediante la factorización de Cholesky. Comprueba su funcionamiento para intentar resolver el sistema $A x=b$ donde $b=\left(2,-5,1 \right)$ en los casos de los ejemplos anteriores:
$$A=\left(\begin{array}{rrr}
2 & 4 & 1 \\
4 & 10 & 1 \\
1 & 1 & 5 \end{array} \right) \qquad 
A=\left(\begin{array}{rrr}
2 & 4 & 1 \\
4 & 0 & 1 \\
1 & 1 & 5 \end{array} \right)$$ 

**No** se permite utilizar la función `linalg.cholesky`de `NumPy`. Hay que modificar el código anterior de la factorización LU para definir una función que haga dicha factorización.


In [39]:

def CHOL_desc(A):
    """
    Esta función calcula la descomposición Cholesky de una matriz A sin intercambio de filas.
    A debe ser un array de Numpy, una matriz simétrica y definida positiva.
    """
    n=np.shape(A)[1]
    L=np.zeros([n,n])
    
    for i in range(n):
        for j in range(i+1):
            suma=0
            if (j==i):
                for k in range(j+1):
                    suma += pow(L[j,k], 2)
                if ( suma <= A[j,j]):
                    L[j,j] = math.sqrt(A[j,j]-suma)
                else: print("ERROR: matriz no definida positiva, descomposición fallida"); return np.zeros([n,n])
            else:
                for k in range(j+1):
                    suma += L[i,k]*L[j,k]
                L[i,j] = (A[i,j]-suma)/L[j,j]
                
    return L


In [40]:
A=np.array([[2,4,1],[4,10,1],[1,1,5]])
b=np.array([2,-5,1])

L=CHOL_desc(A)

y=sust_progresiva(L,b)
x=sust_regresiva(np.transpose(L),y)
print("Solución =", x)
print("Comprobación =", np.linalg.solve(A,b))



Solución = [11.6875 -5.0625 -1.125 ]
Comprobación = [11.6875 -5.0625 -1.125 ]


In [41]:
A=np.array([[2,4,1],[4,0,1],[1,1,5]])
b=np.array([2,-5,1])

L=CHOL_desc(A)
n=np.shape(A)[1]
if (L.any()!=0):
    y=sust_progresiva(L,b)
    x=sust_regresiva(np.transpose(L),y)
    print("Solución =", x)
    print("Comprobación =", np.linalg.solve(A,b))


ERROR: matriz no definida positiva, descomposición fallida
